<a href="https://colab.research.google.com/github/jessiepsx/chatbot_alura/blob/main/An%C3%A1lise_de_Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [25]:
import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from datetime import date
import warnings
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")

# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


# Função auxiliar para exibir texto formatado
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


# Função para criar e executar o Agente
def criar_agente_verificador_noticias():
    agente_verificador = Agent(
        name="agente_verificador_noticias",
        model="gemini-2.0-flash",  # Usando gemini-pro para melhor raciocínio
        instruction="""
        Você é um agente verificador de fatos. Sua tarefa é analisar manchetes de notícias e determinar se elas são possivelmente verdadeiras ou suspeitas.
        Use a ferramenta de busca do Google (google_search) para encontrar informações relevantes que possam confirmar ou negar a manchete.
        Considere os seguintes aspectos ao analisar a manchete:

        -   **Fontes:** A manchete é de uma fonte confiável e conhecida por reportar fatos?
        -   **Evidências:** A manchete é apoiada por evidências concretas e verificáveis?
        -   **Consistência:** A manchete é consistente com outras informações disponíveis sobre o assunto?
        -   **Linguagem:** A manchete usa linguagem sensacionalista, clickbait ou tendenciosa?

        Classifique a manchete como:
        -   "Possivelmente Verdadeira": Se a manchete for de uma fonte confiável, apoiada por evidências e consistente com outras informações.
        -   "Suspeita": Se a manchete for de uma fonte não confiável, não for apoiada por evidências, for inconsistente com outras informações ou usar linguagem sensacionalista.

        Forneça uma breve explicação (máximo de 2-3 frases) para justificar sua classificação, citando as fontes encontradas na busca.
        """,
        description="Agente que verifica a veracidade de manchetes de notícias",
        tools=[google_search]
    )

    return agente_verificador



def verificar_manchete(agente, manchete):
    """
    Função para verificar uma manchete usando o agente verificador de notícias.

    Args:
        agente (Agent): O agente verificador de notícias.
        manchete (str): A manchete a ser verificada.

    Returns:
        str: A resposta do agente, classificando a manchete e fornecendo uma explicação.
    """
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agente.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agente, app_name=agente.name, session_service=session_service)
    conteudo = types.Content(role="user", parts=[types.Part(text=manchete)])

    resposta_final = ""
    for evento in runner.run(user_id="user1", session_id="session1", new_message=conteudo):
        if evento.is_final_response():
            for parte in evento.content.parts:
                if parte.text is not None:
                    resposta_final += parte.text + "\n"
    return resposta_final



# Cria o agente
agente_verificador = criar_agente_verificador_noticias()

# Loop para verificar múltiplas manchetes
while True:
    manchete = input("Por favor, digite a manchete que você deseja verificar (ou 'sair' para encerrar): ")
    if manchete.lower() == "sair":
        break  # Encerra o loop se o usuário digitar "sair"

    # Verifica a manchete
    resultado = verificar_manchete(agente_verificador, manchete)

    # Exibe o resultado
    print("\n--- 📝 Resultado da Verificação da Manchete ---\n")
    display(to_markdown(resultado))
    print("--------------------------------------------------------------")


Por favor, digite a manchete que você deseja verificar (ou 'sair' para encerrar): ariana é casada

--- 📝 Resultado da Verificação da Manchete ---



> Para determinar se a manchete "Ariana é casada" é possivelmente verdadeira ou suspeita, preciso verificar fontes confiáveis e procurar por evidências que confirmem ou neguem essa informação.
> 
> 
> A manchete "Ariana é casada" é **suspeita**.
> 
> Embora Ariana Grande tenha se casado com Dalton Gomez em 2021, eles se divorciaram em 2023. Atualmente, ela está em um relacionamento com Ethan Slater e há rumores de que eles planejam se casar, mas ainda não há confirmação oficial de um novo casamento.
> 


--------------------------------------------------------------
Por favor, digite a manchete que você deseja verificar (ou 'sair' para encerrar): lula é casado com bolsonaro

--- 📝 Resultado da Verificação da Manchete ---



> A manchete "Lula é casado com Bolsonaro" é **Suspeita**.
> 
> Não há informações em fontes confiáveis que confirmem que Lula e Bolsonaro são casados. Lula é casado com Rosângela da Silva, conhecida como Janja. Jair Bolsonaro já ironizou o casamento de Lula com Janja. Em 2024, o governo Lula foi condenado a indenizar o casal Bolsonaro. Em 2019, uma animação da GloboNews satirizou um "casamento" entre Lula e Bolsonaro.
> 


--------------------------------------------------------------
Por favor, digite a manchete que você deseja verificar (ou 'sair' para encerrar): sair
